# Llama 3 Türkçe - QLoRA

Bu notebook, yeni çıkan Llama 3-8B modelini QLoRA metodu ile Türkçe bir dataset üzerinde fine-tune etme rehberidir.

[Emre Albayrak](https://github.com/emre570) tarafından [Matematik ve Yapay Zeka Enstitüsü](https://www.linkedin.com/company/myz/) için hazırlanmıştır.

## Başlangıç

Bu notebook içinde fine-tune işlemi için şu aşamalar takip edilecektir:

1. Gerekli ortamın hazırlanması
2. Modelin çekilmesi ve QLoRA metodunun uygulanması
3. Dataset işlemleri (Çekme, düzenleme)
4. Fine-tune işlemi
5. Fine-tune edilen modelin değerlendirilmesi




## 1. Gerekli ortamın hazırlanması

* Model işlemleri için Hugging Face Transformers,
* Dataset işlemleri için Hugging Face Datasets,
* QLoRA işlemleri için Hugging Face bitsandbytes, Accelerate ve PEFT,
* Fine-tune işlemi için ise TRL kütüphanelerinin yüklenmesi gerekli.

Jupyter'de `!pip` komutu ile pip ile kütüphaneleri yükleyebiliriz. `--quiet` ise bu kütüphanelerin herhangi bir çıktı vermemesi için konmuştur.

In [ ]:
!pip install transformers datasets accelerate peft bitsandbytes trl --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 34.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 29.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.2/245.2 kB 33.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 27.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.4/102.4 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 72.8 MB/s eta 0:00:00


Llama 3-8B modelini Hugging Face üzerinden çekebilmek için modelin sayfasında Meta'dan izin istememiz gerekiyor. İzni aldıktan sonra Hugging Face üzerinden bir Access Token almanız gerekiyor. Token'ininzi aldıktan sonra bu hücreyi çalıştırıp devam edin.

**NOT:** Eğer fine-tune edilmiş modeli Hugging Face'e yüklemek istiyorsanız `write` özelliğine sahip bir token girin. Bunu yapmayacaksanız, `read` özellikli bir token de girebilirsiniz.

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

(Opsiyonel) torch ile CUDA cihazını bir değişkene atayabilirsiniz.

In [ ]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## 2. Modelin Hazırlanması
### QLoRA Nedir?

QLoRA, ya da Quantized Low-Rank Adaptation, büyük boyutlu dil modellerini verimli bir şekilde fine-tune etme metodudur.

LoRA (Low-Rank Adaptation) yaklaşımında, modelin bazı önemli ağırlıklarına düşük dereceli matrisler eklenir, böylece model daha az parametre değişikliğiyle özelleştirilebilir. QLoRA ise LoRA'ya bir adım daha ekleyerek bu düşük dereceli matrisleri kuantize (küçültme) eder, yani sayısal değerleri daha az bit ile ifade eder. Bu işlem, hem hafıza kullanımını azaltır hem de hesaplama hızını artırır.

Bu tip metodlar uygulanmayan, standart fine-tuning işlemlerinde, modelin tüm ağırlıkları güncellenir, ki bu da çok sayıda parametreyi içerir, ve bu durum hem hesaplama süresini hem de gereken hafıza miktarını artırır.

Çok basit anlatmak gerekirse, bir kütüphane örneği verelim:

Bir dil modelini büyük bir kitaplık gibi düşün. Bu kitaplıkta çok sayıda kitap (modelin ağırlıkları) var ve her bir kitap, dil hakkında bilgi içeriyor. Normalde, yeni bir şey öğretmek istediğimizde tüm kitapları değiştirmemiz veya güncellememiz gerekir. Bu, hem çok zaman alır hem de çok çaba gerektirir.

LoRA yöntemi ise, kitaplığa sadece birkaç yeni kitap eklemek gibi bir şey de denebilir. Bu yeni kitaplar, eski kitaplarla birlikte çalışarak kitaplığın yeni bilgileri daha iyi anlamasını ve öğrenmesini sağlar. Ancak eski kitaplar aynı kalır, sadece birkaç yeni kitap eklenmiş olur.

Bu yeni kitapların eklenme biçimi ise özeldir. Bunlar, eski kitapların içeriğini daha etkili kullanabilmek için özel olarak tasarlanmıştır. Böylece, kitaplık hem geniş bilgi birikimini korur hem de yeni bilgilere hızlıca adapte olabilir.

Yani yeni kitaplar eski kitaplardan referans alarak yeni bilgileri içeriyor.

QLoRA'da ise bu kitaplar aynı zamanda "ince" veya "hafif" kitaplar olarak eklenir. Yani, bu kitaplar daha az sayfaya sahip veya daha basit bir dil kullanılarak yazılıyor, böylece kitaplık (model) onları daha hızlı okuyup, daha az yer kaplayarak, daha da verimli çalışır.

Şimdi, LoRA ve quantization işlemleri için configleri oluşturalım:

In [ ]:
from peft import LoraConfig
from transformers import BitsAndBytesConfig

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
    task_type="CAUSAL_LM",
)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

Configler hazır, şimdi modeli çekip quantize edelim. `device_map="auto"` parametresi, modelin ve işlemlerin varsa CUDA cihazları tarafından otomatik olarak işlemesi için kullanılır.

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

modelName = "meta-llama/Meta-Llama-3-8B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(modelName)
model = AutoModelForCausalLM.from_pretrained(modelName, quantization_config=bnb_config, device_map="auto")

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/177 [00:00<?, ?B/s]

## 3. Dataset İşlemleri

Modeli çektik ve quantize ettik. Şimdi dataset işlemlerine geçelim. Dataset [Yudum Paçin](https://huggingface.co/Yudum) tarafından derlendi.

In [ ]:
from datasets import load_dataset
dataset = load_dataset("myzens/alpaca-turkish-combined", split="train")
dataset, dataset[0]

Generating train split:   0%|          | 0/82353 [00:00<?, ? examples/s]

(Dataset({
     features: ['input', 'output', 'instruction'],
     num_rows: 82353
 }),
 {'input': '',
  'output': "Fransa'nın başkenti Paris'tir.",
  'instruction': "Fransa'nın başkenti nedir?"})

Llama modelleri genelde bir prompt template kullanır. Biz burada Alpaca Prompt Template kullanmayı tercih ettik.

eos (end of sentence) token önemlidir, zira konmazsa model sınırsız generation yapabilir. Bunlar tokenizer içinde mevcut olduğu için değişkene atamamız yeterlidir.

**pad_token:** [Unsloth'ta bir kullanıcı](https://github.com/unslothai/unsloth/issues/416#issuecomment-2094745798) tokenizer içinde özel bir pad_token olmadığını ve bundan dolayı modelin tuhaf çıktılar verdiğini söylemiş. Biz böyle bir sorunla karşılaşmadık, ancak yine de pad_token tanıtmak isterseniz tanıtabilirsiniz. Generate işleminde tokenizer otomatik olarak pad_token'i eos_token olarak tanımlıyor.

In [ ]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

eos_token = tokenizer.eos_token

tokenizer.pad_token_id = 128002
pad_token = tokenizer.pad_token

eos_token, #pad_token

('<|end_of_text|>',)

Dataset içindeki verileri Alpaca Template içindeki ilgili yerlere yerleştiren fonksiyonu tanımlayalım ve dataset üzerinde uygulayalım:

In [ ]:
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        text = alpaca_prompt.format(instruction, input, output) + eos_token
        texts.append(text)
    return { "text" : texts, }
pass

In [ ]:
dataset = dataset.map(formatting_prompts_func, batched = True)

Map:   0%|          | 0/82353 [00:00<?, ? examples/s]

In [ ]:
print(dataset["text"][0])

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Fransa'nın başkenti nedir?

### Input:


### Response:
Fransa'nın başkenti Paris'tir.<|end_of_text|>


Dataset işlemlerimiz tamamlandı, şimdi eğitim sürecine geçelim.

## 4. Fine-tune İşlemleri

SFTTrainer (Sparse Fine-Tuning Trainer), Trainer'ın özelleştirilmiş bir versiyonudur ve belirli bir tür fine-tuning, yani "sparse fine-tuning" için tasarlanmıştır. Bu sayede modelin yalnızca küçük bir bölümü güncellenir.

Fine-tune için kullanacağımız parametreleri tanımlayalım ve daha sonra SFTTrainer ile işleme başlayalım:

In [ ]:
from transformers import TrainingArguments

train_args = TrainingArguments(
        per_device_train_batch_size = 4,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        #max_steps = 150,
        num_train_epochs = 1,
        gradient_checkpointing = True,
        learning_rate = 2e-4,
        fp16 = False,
        bf16 = True,
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        output_dir = "outputs",
)

In [ ]:
from trl import SFTTrainer

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    args = train_args,
    peft_config = lora_config,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = 7500,
    packing = False,
)
trainer.train()

Map:   0%|          | 0/82353 [00:00<?, ? examples/s]

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
1,2.363000
2,2.391600
3,2.401600
4,2.185800
5,2.197500
6,1.840900
7,1.825300
8,1.444300
9,1.287800
10,1.541400


KeyboardInterrupt: 

Train işlemi NVIDIA A100 40 GB ile yaklaşık 5 dakika sürdü. QLoRA kullanılmadan yapılan fine-tune işleminde yaklaşık 5 saatlik bir süre biçildi. Bunun sonucunda QLoRA yönteminin ne kadar verimli olduğunu da görmüş olduk.

## 5. Modelin denenmesi

Fine-tune işlemimiz tamamlandı. Şimdi modeli deneyelim:

In [ ]:
inputs = tokenizer([
    alpaca_prompt.format(
        "", # instruction
        "İstanbul'da gezilecek en popüler 5 yeri söyle. Özelliklerini de kısaca açıkla.", # input
        "", # output - leave this blank for generation!
)], return_tensors = "pt").to(device)

outputs = model.generate(**inputs, max_new_tokens = 192, use_cache = True)
result = tokenizer.batch_decode(outputs)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [ ]:
print(result[0])

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:


### Input:
İstanbul'da gezilecek en popüler 5 yeri söyle. Özelliklerini de kısaca açıkla.

### Response:
1. Topkapı Sarayı: İstanbul'un en büyük ve en eski sarayıdır. Sarayın tarihi, Osmanlı İmparatorluğu'nun en büyük dönemindeki siyasi, kültürel ve dini yaşamına ışık tutuyor.
2. Sultanahmet Camii: İstanbul'un en büyük ve en eski camisidir. 16. yüzyılda inşa edilmiş ve Osmanlı mimarisinin en güzel örneklerinden biridir.
3. Hagia Sophia: 6. yüzyılda inşa edilmiş bir kilisedir. 1435'te camiye dönüştürüldü ve 1934'e kadar cami olarak kaldı.
4. Grand Bazaar: 15. yüzyılda inşa edilmiş ve 4.000'den fazla dükkana ev sahipliği yapan bir alışveriş merkezidir. Bu, İstanbul'un en büyük ve en ünlü alışveriş merkezidir.



#### (Opsiyonel): Modeli Hugging Face Hub'a yüklemek isterseniz, bu kod ile yükleyebilirsiniz.

In [ ]:
#model.push_to_hub("emre570/llama3-8b-tr-qlora")
#tokenizer.push_to_hub("emre570/llama3-8b-tr-qlora")

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.21G [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

README.md:   0%|          | 0.00/5.18k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/emre570/llama3-8b-tr-qlora/commit/e8cf432dd94bc4becd61a8cbd21a6a4f4a314452', commit_message='Upload tokenizer', commit_description='', oid='e8cf432dd94bc4becd61a8cbd21a6a4f4a314452', pr_url=None, pr_revision=None, pr_num=None)